Este notebook contiene las consultas realizadas en SQL sobre una base de datos relacional. Si bien dichas consultas se ejecutaron originalmente en Amazon Redshift y se utilizarán los tiempos obtenidos allí para la comparación, este notebook se incluye como complemento para ilustrar cómo se puede trabajar con una base de datos relacional directamente desde un entorno de Google Colab.

Además, permite comparar los resultados obtenidos con los de la base de datos documental, utilizando los mismos archivos de origen que se emplearon para construir las colecciones en dicha base.

In [1]:
!pip install pandas sqlalchemy sqlite-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.7 MB/s eta 0:00:00


Subir los archivos a la session de collab antes de correr la siguiente celda.

In [3]:
#cargamos los archivos a dataframes de pandas

import pandas as pd

dim_client = pd.read_json('/content/dim_client.json')
dim_date = pd.read_json('/content/dim_date.json')
dim_device = pd.read_json('/content/dim_device.json')
dim_location = pd.read_json('/content/dim_location.json')
dim_media = pd.read_json('/content/dim_media.json')
dim_partner = pd.read_json('/content/dim_partner.json')
dim_platform = pd.read_json('/content/dim_platform.json')
fact_vod_uu_details = pd.read_json('/content/fact_vod_uu_details.json')


In [4]:
#creamos una base de datos SQLite in-memory
import sqlite3

conn = sqlite3.connect(":memory:")  # in-memory DB


In [5]:
#escribimos dataframes a tablas SQLite

dim_client.to_sql('dim_client', conn, index=False, if_exists='replace')
dim_date.to_sql('dim_date', conn, index=False, if_exists='replace')
dim_device.to_sql('dim_device', conn, index=False, if_exists='replace')
dim_location.to_sql('dim_location', conn, index=False, if_exists='replace')
dim_media.to_sql('dim_media', conn, index=False, if_exists='replace')
dim_partner.to_sql('dim_partner', conn, index=False, if_exists='replace')
dim_platform.to_sql('dim_platform', conn, index=False, if_exists='replace')
fact_vod_uu_details.to_sql('fact_vod_uu_details', conn, index=False, if_exists='replace')


215347

Queries relacionales

Query 1: Top 10 contenidos del
primero al 10 de mayo
según total de tiempo
visto

In [6]:
fact_vod_uu_details.columns


Index(['sessions', 'device_type_name', 'category_name', 'partner_name',
       'minutes_watched', 'seconds_watched', 'client_id', 'video_type',
       'country_code', 'partner_id', 'device_type_id', 'country_name',
       'platform_id', 'platform_name', 'media_id', 'media_name',
       'hours_watched', 'uuid_hashvalue', 'date_code', 'full_date',
       'client_name'],
      dtype='object')

In [16]:
# SQL query
query_1 = """
SELECT
    dm.video_type,
    dm.media_name,
    SUM(fvd.seconds_watched) AS total_seconds
FROM
    fact_vod_uu_details AS fvd


INNER JOIN dim_media AS dm
    ON fvd.media_id = dm.media_id

GROUP BY
    dm.video_type, dm.media_name
ORDER BY
    total_seconds DESC
LIMIT 10;
"""


# Run query and show result
result_df = pd.read_sql_query(query_1, conn)
result_df


,video_type,media_name,total_seconds
0,Feature,My Little Pony Equestria Girls: Rainbow Rocks,3542125
1,Episode,Fire-Breathing Magic Dinosaur,876877
2,Episode,Merry-go-round,867021
3,Feature,Diamond Dog Caper,830416
4,Episode,Beat the Raccoon,636846
5,Episode,Pirates of Love,614098
6,Episode,Magic wand,588624
7,Feature,End of Year Special - Bubu and the Little Owls,571157
8,Episode,The Missing Hero Crystal,553976
9,Episode,Holidays Forever Compilation,551834


In [17]:
import time

start_time = time.time()

# Run query
result_df = pd.read_sql_query(query_1, conn)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Query executed in {elapsed_time:.4f} seconds")


Query executed in 0.5370 seconds


Query 2: Tiempo total visto por
cliente en la última
semana

In [18]:
query_2 = """
SELECT
    dc.client_name,
    SUM(fvd.seconds_watched) AS total_seconds
FROM
    fact_vod_uu_details AS fvd
INNER JOIN dim_client AS dc
    ON fvd.client_id = dc.client_id

GROUP BY
    dc.client_name
ORDER BY
    total_seconds DESC;
"""

# Execute and show results
result_df = pd.read_sql_query(query_2, conn)
result_df


,client_name,total_seconds
0,Toon Goggles,92363525


In [ ]:

start_time = time.time()

# Execute query
result_df = pd.read_sql_query(query_2, conn)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Query executed in {elapsed_time:.4f} seconds")


Query executed in 0.2302 seconds


Query 3: Cantidad de Usuarios que
vieron contenido por día

In [20]:
query_3 = """
SELECT
    fvd.full_date,
    COUNT(DISTINCT fvd.uuid_hashvalue) AS total_users
FROM
    fact_vod_uu_details AS fvd

INNER JOIN dim_client AS c
    ON fvd.client_id = c.client_id
INNER JOIN dim_media AS m
    ON fvd.media_id = m.media_id
WHERE
    c.client_id = 'tg'
    AND m.video_type != 'Linear'
GROUP BY
    fvd.full_date
ORDER BY
    fvd.full_date ASC;
"""

# Execute and show results
result_df = pd.read_sql_query(query_3, conn)
result_df


,full_date,total_users
0,2025-05-01,5729
1,2025-05-02,4530
2,2025-05-03,4866
3,2025-05-04,4546
4,2025-05-05,3957
5,2025-05-06,3914
6,2025-05-07,4009
7,2025-05-08,4092
8,2025-05-09,4189
9,2025-05-10,4843


In [ ]:
start_time = time.time()

# Execute query
result_df = pd.read_sql_query(query_3, conn)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Query executed in {elapsed_time:.4f} seconds")


Query executed in 0.4719 seconds


Query 4: Tiempo total visto por
plataforma y cliente
(principales plataformas
utilizadas)

In [25]:
query_4 = """
SELECT
    fvd.client_name,
    fvd.partner_name,
    SUM(fvd.seconds_watched) AS total_seconds
FROM
    fact_vod_uu_details AS fvd

GROUP BY
    fvd.client_name, fvd.partner_name
ORDER BY
    total_seconds DESC
    LIMIT 10;
"""

# Execute and show results
result_df = pd.read_sql_query(query_4, conn)
result_df


,client_name,partner_name,total_seconds
0,Toon Goggles,Hisense,27459163
1,Toon Goggles,Roku,13100312
2,Toon Goggles,Vizio Smartcast,9557878
3,Toon Goggles,TCL,9469610
4,Toon Goggles,M Star,7468756
5,Toon Goggles,General Android,4024805
6,Toon Goggles,Philips,3668155
7,Toon Goggles,Unknown,2898904
8,Toon Goggles,General HTML5,1845031
9,Toon Goggles,Hisense Legacy,1665119


In [ ]:
start_time = time.time()

# Execute query
result_df = pd.read_sql_query(query_4, conn)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Query executed in {elapsed_time:.4f} seconds")

Query executed in 76.6030 seconds


comparacion adicional - comparacion de tamano y almacenamiento

Medir el tamaño de la base SQLite
¿Qué podemos analizar o concluir?

¿MongoDB ocupa más por duplicación o menos por no normalizar?

¿Hay diferencia notable entre tipos de datos usados?

¿Vale la pena ese aumento en tamaño si el modelo es más práctico?

In [ ]:
# Guardar la base de datos en un archivo físico
disk_conn = sqlite3.connect("/content/sqlite_db.db")
with disk_conn:
    conn.backup(disk_conn)

# Medir el tamaño en bytes
import os

sqlite_size = os.path.getsize("/content/sqlite_db.db") / (1024 * 1024)  # en MB
print(f"Tamaño del archivo SQLite: {sqlite_size:.2f} MB")


Tamaño del archivo SQLite: 46.89 MB
